In [19]:
# imports

import numpy as np
import pandas as pd
import flopy
import flopy.utils.binaryfile as bf
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import shutil
import os

In [20]:
flopy.__version__

'3.9.5'

# Nombre del modelo

In [21]:
# nombre del modelo y workspace
modelname = "EjemploFlopy"
workspace = "Model"
mf = flopy.modflow.Modflow(modelname, model_ws = workspace, exe_name = "bin/mf2005")

# Discretizacion

## Espacial

In [22]:
# Discretizacion espacial del modelo
Lx = 2000 # largo en x
Ly = 1500 # largo en y
ztop = 200 # profundidad de la superficie
zbot = -200 # profundidad de la base
nlay = 4 # numero de capas
nrow = 15 # numero de filas
ncol = 20 # numero de columnas
delr = Lx/ncol # tamaño de celda en x
delc = Ly/nrow # tamaño de celda en y
delv = (ztop - zbot)/nlay # tamaño de celda en z
botm = np.linspace(ztop, zbot, nlay+1) # 

## Temporal

In [23]:
# Discretización temporal del modelo
nper = 4 # numero de periodos
perlen = [10., 5., 5., 10.] # tiempo de cada periodo en dias
perlen = [x * 86400 for x in perlen] # tiempo de cada periodo en segundos
nstp = [5,2,2,3] # numero de pasos de tiempo en cada periodo
steady = False # modelo transitorio
itmuni=1 # unidad de tiempo en segundos
lenuni=2 # unidad de longitud en metros


## Paquete DIS

In [24]:
# genera el paquete DIS
dis = flopy.modflow.ModflowDis(mf, nlay, nrow, ncol, delr=delr, delc=delc, top=ztop, botm=botm[1:],
                               lenuni=lenuni, itmuni=itmuni, nper=nper, perlen=perlen, nstp=nstp, steady=steady)

## Paquete BAS

In [25]:
# paquete bas con CH
ibound = np.ones((nlay, nrow, ncol), dtype=np.int32)
ibound[0, :, 0]  = -1 #primera capa, todas las filas, primera columna
ibound[0, :, -1] = -1 # primera capa, todas las filas, ultima columna
strt = np.ones((nlay, nrow, ncol), dtype=np.float32)
strt[0, :, 0]  = 100 # primera capa, todas las filas, primera columna
strt[0, :, -1] = 60 # primera capa, todas las filas, ultima columna
bas = flopy.modflow.ModflowBas(mf, ibound=ibound, strt=strt)

## Paquete WEL

In [26]:
# crea el sp_data de un pozo para 4 periodos
sp_data = {0: [2,3,11,-0.015], 1: [2,3,11,-0.03], 2: [2,3,11,-0.04], 3: [2,3,11,0]} # capa, fila, columna, caudal en m3/s
wel = flopy.modflow.ModflowWel(mf, stress_period_data=sp_data)

## LPF

In [27]:
# definir conductividad hidraulica
hk  = 3e-5 # conductividad hidraulica horizontal (m3/s)
vka = 1e-5 # conductividad hidraulica vertical (m3/s)
lpf = flopy.modflow.ModflowLpf(mf, hk=hk, vka=vka)

## OC

In [28]:
# output control
stress_period_data = {(0,4):['save head'], (1,1) : ['save head'], (2,1) : ['save head'], (3,2) : ['save head']} # periodo, etapa, head
oc = flopy.modflow.ModflowOc(mf, stress_period_data=stress_period_data)


## Solver

In [29]:
pcg = flopy.modflow.ModflowPcg(mf)

## Ejecucion

In [30]:
# escribir el archivo de entrada
mf.write_input()

# RUN

In [31]:
success, buff = mf.run_model()

FloPy is using the following executable to run the model: ..\bin\mf2005.exe

                                  MODFLOW-2005     
    U.S. GEOLOGICAL SURVEY MODULAR FINITE-DIFFERENCE GROUND-WATER FLOW MODEL
                             Version 1.12.00 2/3/2017                        

 Using NAME file: EjemploFlopy.nam 
 Run start date and time (yyyy/mm/dd hh:mm:ss): 2025/10/04 12:14:01

 Solving:  Stress period:     1    Time step:     1    Ground-Water Flow Eqn.
 Solving:  Stress period:     1    Time step:     2    Ground-Water Flow Eqn.
 Solving:  Stress period:     1    Time step:     3    Ground-Water Flow Eqn.
 Solving:  Stress period:     1    Time step:     4    Ground-Water Flow Eqn.
 Solving:  Stress period:     1    Time step:     5    Ground-Water Flow Eqn.
 Solving:  Stress period:     2    Time step:     1    Ground-Water Flow Eqn.
 Solving:  Stress period:     2    Time step:     2    Ground-Water Flow Eqn.
 Solving:  Stress period:     3    Time step:     1    Ground-Wa

In [ ]:
# convertir hds a bhd para poder importar a modelmuse
shutil.copyfile(os.path.join(workspace, modelname+'.hds'), os.path.join(workspace, modelname+'.bhd'))

'Model\\EjemploFlopy.bhd'

## Plots

## 